# Exercise 2

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import math
import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 28 * 28], name='input_images')
y_ = tf.placeholder(tf.float32, [None, 10], name='labels')
drop_prob = tf.placeholder(tf.float32, name='drop_prob')

tf.summary.image('input', tf.reshape(x, [-1, 28, 28, 1]))
W1 = tf.Variable(tf.truncated_normal([28 * 28, 512], stddev=1 / math.sqrt(28 * 28)), name='L1_weights')
b1 = tf.Variable(tf.constant(0.1, shape=[512]), name='L1_biases')
h1 = tf.nn.relu(tf.matmul(x, W1) + b1, name='L1_ReLU')
h1_drop = tf.nn.dropout(h1, 1.0 - drop_prob, name='L1_dropout')
tf.summary.histogram('L1_weights', W1)

W2 = tf.Variable(tf.truncated_normal([512, 256], stddev=1 / math.sqrt(512)), name='L2_weights')
b2 = tf.Variable(tf.constant(0.1, shape=[256]), name='L2_biases')
h2 = tf.nn.relu(tf.matmul(h1_drop, W2) + b2, name='L2_ReLU')
h2_drop = tf.nn.dropout(h2, 1.0 - drop_prob, name='L2_dropout')
tf.summary.histogram('L2_weights', W2)

W3 = tf.Variable(tf.truncated_normal([256, 10], stddev=1 / math.sqrt(256)), name='L3_weights')
b3 = tf.Variable(tf.constant(0.1, shape=[10]), name='L3_biases')
logits = tf.matmul(h2_drop, W3) + b3
y = tf.nn.softmax(logits, name='softmax')
tf.summary.histogram('L3_weights', W3)
tf.summary.histogram('logits', logits)
tf.summary.histogram('output', y)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), axis=1), name='cross_entropy')
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
tf.summary.scalar('loss', cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1), name='correct_prediction')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='train_accuracy')
tf.summary.scalar('accuracy', accuracy)

init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
merged_summaries = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter('./summary/dropout', sess.graph)
sess.run(init)
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    _, summary = sess.run([train_step, merged_summaries], feed_dict={x: batch_xs, y_: batch_ys, drop_prob: 0.4})
    summary_writer.add_summary(summary, i)

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, drop_prob: 0.0}))

0.9695
